In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras
from sklearn.preprocessing import MinMaxScaler


In [2]:
file_path = 'https://docs.google.com/spreadsheets/d/1V6cBNRohsvWOEcPUMWyZ6tMAvc25Ji2U/export?format=csv'
data = pd.read_csv(file_path)


In [3]:

# Ensure all values are numeric (convert non-numeric values to NaN)
data = data.apply(pd.to_numeric, errors='coerce').fillna(0)
# Remove rows where all values are zero
data = data.loc[~(data.eq(0).all(axis=1))]

data['resi_value'] = data.apply(lambda row : row['resi_value'] if row['trend_type']==-1 else 100-row['resi_value'], axis = 1)

data.tail(30)



,time,max_seen,min_seen,trend_type,price,leg_3_to_1,leg_2_to_1,leg_2_correction,resistance_filter,rsi_filter,resi_value,max_move
19602,0.0,1.08191,1.08088,1,1.08173,1.37,0.33,71,0,0,25.40,18
19603,0.0,1.08191,1.08108,-1,1.08111,0.60,0.63,38,1,1,32.46,3
19604,0.0,1.08145,1.08109,-1,1.08111,0.46,0.44,87,1,0,27.67,2
19605,0.0,1.08204,1.08056,-1,1.08118,0.85,0.58,86,1,0,26.32,62
19606,0.0,1.08263,1.08052,1,1.08167,4.13,0.96,63,1,1,33.13,96
19607,0.0,1.08263,1.07688,-1,1.07975,0.80,0.18,24,0,1,38.69,287
19608,0.0,1.08590,1.08099,1,1.08175,0.40,0.06,36,0,1,31.41,415
19609,0.0,1.08590,1.08114,1,1.08163,1.25,0.47,41,0,1,37.68,427
19610,0.0,1.08263,1.08130,-1,1.08181,1.64,0.43,96,0,1,35.31,51
19611,0.0,1.08259,1.08130,-1,1.08172,0.94,0.22,23,0,1,34.15,42


In [4]:
# # replace all negative values in max_price_move with 0 because out stoploss has been hitted anyway
# data["max_price_move"] = data["max_price_move"].apply(lambda x: max(0, x))
# data

In [5]:
reserve = data
data = data.drop(['time','price','trend_type','max_seen', 'min_seen'], axis=1)
# data = data.drop(['rsi_filter','resistance_filter'], axis=1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19632 entries, 0 to 19631
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   leg_3_to_1         19632 non-null  float64
 1   leg_2_to_1         19632 non-null  float64
 2   leg_2_correction   19632 non-null  int64  
 3   resistance_filter  19632 non-null  int64  
 4   rsi_filter         19632 non-null  int64  
 5   resi_value         19632 non-null  float64
 6   max_move           19632 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 1.2 MB


In [7]:
data.describe()

,leg_3_to_1,leg_2_to_1,leg_2_correction,resistance_filter,rsi_filter,resi_value,max_move
count,19632.000000,19632.000000,19632.000000,19632.000000,19632.000000,19632.000000,19632.00000
mean,1.653643,0.871775,60.405410,0.313162,0.606764,31.333625,657.90378
std,3.023404,2.029656,21.848241,0.463792,0.488481,6.301682,1735.23416
min,0.010000,0.010000,7.000000,0.000000,0.000000,4.420000,-103.00000
25%,0.530000,0.270000,43.000000,0.000000,0.000000,27.380000,47.00000
50%,1.030000,0.510000,60.000000,0.000000,1.000000,31.770000,150.00000
75%,1.920000,0.950000,78.000000,1.000000,1.000000,35.800000,506.00000
max,162.000000,92.000000,100.000000,1.000000,1.000000,65.740000,21468.00000


In [8]:
data.shape

(19632, 7)

In [9]:
data.isnull().sum()

leg_3_to_1           0
leg_2_to_1           0
leg_2_correction     0
resistance_filter    0
rsi_filter           0
resi_value           0
max_move             0
dtype: int64

In [10]:
# normalized_df = keras.utils.normalize(data)
# type(normalized_df)
# data = pd.DataFrame(normalized_df, columns=["leg_3_to_leg_1_time","leg_2_to_leg_1_time","leg_2_correction_percent","resistance","rsi_filter_passed","normalized_rsi","max_price_move"])
# data

In [11]:
#normalize data
columns_to_normalize = data.columns.difference(['rsi_filter'])
scaler = MinMaxScaler()
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

#print(data)



In [ ]:
import neat
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Example data setup, replace with your actual dataset
data = pd.DataFrame({
    'leg_3_to_1': np.random.randn(100),
    'leg_2_to_1': np.random.randn(100),
    'leg_2_correction': np.random.randn(100),
    'resistance_filter': np.random.randn(100),
    'rsi_filter': np.random.randn(100),
    'resi_value': np.random.randn(100),
    'max_move': np.random.randn(100)
})

# Separate features and target variable
X = data[['leg_3_to_1', 'leg_2_to_1', 'leg_2_correction', 'resistance_filter', 'rsi_filter', 'resi_value']]
y = data['max_move']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the NEAT fitness function
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        # Convert NEAT genome to TensorFlow model
        model = create_tf_model_from_genome(genome, config, input_shape=X_train_scaled.shape[1])

        # Compile the model
        model.compile(optimizer='adam', loss='mse')

        # Train the model
        model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

        # Evaluate and set the fitness score
        predictions = model.predict(X_test_scaled)
        mse = np.mean((predictions.flatten() - y_test) ** 2)
        genome.fitness = -mse  # Using negative MSE to maximize fitness

# Function to create a TensorFlow model from a NEAT genome
def create_tf_model_from_genome(genome, config, input_shape):
    model = tf.keras.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.InputLayer(input_shape=(input_shape,)))

    # Hidden layers based on genome connections
    for node_key, node in genome.nodes.items():
        if node_key not in config.genome_config.output_keys:
            activation = 'relu' if node.bias > 0 else 'sigmoid'
            model.add(tf.keras.layers.Dense(units=10, activation=activation))
    
    # Output layer
    model.add(tf.keras.layers.Dense(1))  # Regression output

    return model

# Load NEAT configuration
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config-feedforward')

# Initialize population and evolution reporters
population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)

# Run NEAT
winner = population.run(eval_genomes, n=20)

# Evaluate the best evolved model
best_model = create_tf_model_from_genome(winner, config, input_shape=X_train_scaled.shape[1])
best_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
best_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)
test_loss, test_mae = best_model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test MAE of best evolved model: {test_mae}")


NameError: name 'neat' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

# Mean Absolute Error
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.show()


In [ ]:
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


In [ ]:
import numpy as np

# Make predictions
predictions = model.predict(X_test_scaled).flatten()

# Compare predictions with actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(comparison_df.head(10))  # Display first 10 rows for a quick check
